# Align VEHICLe Fragments to BRD4 Inhibitors

In [10]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [11]:
method = "molgrid"

In [12]:
import open3d as o3d
from rdkit import Chem
import numpy as np

from collections import defaultdict

import tqdm
import os, sys, re

import pickle

In [13]:
sys.path.append("../../")
sys.path.append("../")

In [14]:
from utils import align, show

## Load BRD4 Inhibitors

In [15]:
path = "../ligands/BRD4"

In [16]:
files = []
for f in os.listdir(path):
    fname, ext = os.path.splitext(f)
    if (
        ext == ".pcd"
        and method in fname
        and not "tran" in fname
        and not "murcko" in fname
    ):
        files.append(os.path.join(path, f))

# Sort BRD4 ligand files by number
r = re.compile("\d{1,2}")
files.sort(key=lambda f: int(r.search(os.path.basename(f)).group()))

print(files)

pcds = []
mols = []
for f in files:
    pcd = o3d.io.read_point_cloud(f)
    pcds.append(pcd)

    s = Chem.SDMolSupplier(f.replace(".pcd", ".sdf").replace(f"_{method}", ""))
    mol = next(s)
    mols.append(mol)

['../ligands/BRD4/ligand-1_molgrid.pcd', '../ligands/BRD4/ligand-2_molgrid.pcd', '../ligands/BRD4/ligand-3_molgrid.pcd', '../ligands/BRD4/ligand-4_molgrid.pcd', '../ligands/BRD4/ligand-5_molgrid.pcd', '../ligands/BRD4/ligand-6_molgrid.pcd', '../ligands/BRD4/ligand-7_molgrid.pcd', '../ligands/BRD4/ligand-8_molgrid.pcd', '../ligands/BRD4/ligand-9_molgrid.pcd', '../ligands/BRD4/ligand-10_molgrid.pcd']


## Load Fragments

Load fragments. Only fragments that are considered to be synthetically accessible are considered here, in order to reduce computational resources:

In [17]:
fragpath = path = "../../fragments/VEHICLe-good/"
fragfiles = [
    os.path.join(fragpath, f)
    for f in os.listdir(fragpath)
    if os.path.splitext(f)[-1] == ".pcd"
]

print(fragfiles[:5])

fragpcds = []
fragmols = []
for ff in tqdm.tqdm(fragfiles):
    pcd = o3d.io.read_point_cloud(ff)
    fragpcds.append(pcd)

    s = Chem.SDMolSupplier(ff.replace(".pcd", ".sdf"))
    mol = next(s)
    fragmols.append(mol)

['../../fragments/VEHICLe-good/fragment_1046.pcd', '../../fragments/VEHICLe-good/fragment_0.pcd', '../../fragments/VEHICLe-good/fragment_1.pcd', '../../fragments/VEHICLe-good/fragment_1047.pcd', '../../fragments/VEHICLe-good/fragment_10.pcd']


100%|██████████| 5419/5419 [00:05<00:00, 982.31it/s] 


## Align Fragments to CDK2 Inhibitors

In [18]:
alignments = {}

for i, f in enumerate(files):
    name = os.path.splitext(os.path.basename(f))[0].replace(f"_{method}", "")

    alignments[name] = defaultdict(list)

    for j, ff in enumerate(tqdm.tqdm(fragfiles, desc="Fragments " + name)):
        fidx = os.path.splitext(os.path.basename(ff))[0].replace(f"fragment_", "")

        try:
            gfit, cfit, hfit = align(fragmols[j], fragpcds[j], pcds[i], hfit=True)
        except RuntimeError:  # No alignment found
            gfit, cfit, hfit = np.nan

        alignments[name]["fragment"].append(int(fidx))
        alignments[name]["gfit"].append(gfit.fitness)
        alignments[name]["cfit"].append(cfit.fitness)
        alignments[name]["hfit"].append(hfit)
        alignments[name]["mol"].append(mols[i])
        alignments[name]["fragmol"].append(fragmols[j])

Fragments ligand-1:  22%|██▏       | 1179/5419 [10:01<43:16,  1.63it/s] 

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


Fragments ligand-1:  50%|████▉     | 2699/5419 [26:33<25:44,  1.76it/s]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


Fragments ligand-1:  50%|█████     | 2730/5419 [26:53<25:19,  1.77it/s]

...

Fragments ligand-10:  54%|█████▍    | 2948/5419 [27:07<22:55,  1.80it/s]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


Fragments ligand-10:  67%|██████▋   | 3611/5419 [33:16<15:13,  1.98it/s]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


Fragments ligand-10: 100%|██████████| 5419/5419 [50:05<00:00,  1.80it/s]


In [19]:
with open("BRD4-VEHICLe.pkl", "wb") as f:
    pickle.dump(alignments, f)